<a href="https://colab.research.google.com/github/Y-Kanan/Time-Series-classification/blob/master/test_singlePairTime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}



E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [73]:
import matplotlib.pyplot as plt
import copy
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd
from collections import Counter
from sklearn import preprocessing
import datetime

# 105*144 plane
train_downloaded = drive.CreateFile({'id': '1MNstYEkQwv9fB9clNjdtq-uoocK3iNmx'})
train_downloaded.GetContentFile('Plane_TRAIN.csv')
test_downloaded = drive.CreateFile({'id': '1EQPiIAwK44KuuGjqoh7tZHIO87uLAhKl'})
test_downloaded.GetContentFile('Plane_TEST.csv')
# 与平常read_csv无异
train_data = pd.read_csv('Plane_TRAIN.csv',header=None)
test_data = pd.read_csv('Plane_TEST.csv',header=None)

z_train=preprocessing.scale(train_data.iloc[:,1:],axis=1) #axis=1,以行为单位运算
train_data.iloc[:,1:]=z_train
z_test=preprocessing.scale(test_data.iloc[:,1:],axis=1)
test_data.iloc[:,1:]=z_test

x=z_train[0]
y=z_test[0]
# print(x,'\n',y)



# ---------------------定义的函数部分----------------------------------------------------------------------------------------

def reduce(x,n):
    if n==1:
        return x
    x_shrinked=[(x[i] + x[1 + i]) / n for i in range(0, len(x) - len(x)%n - 1, n)]
    x_shrinked=np.array(x_shrinked)
    return x_shrinked

  
#基于差分的DTW改进
def DDTW(x,y):      
    x_len=np.size(x)
    
#     差分处理
    for i in range(1,x_len-1):
        x[i] = ((x[i] - x[i - 1]) + (x[i + 1] - x[i - 1]) / 2) / 2
        y[i] = ((y[i] - y[i - 1]) + (y[i + 1] - y[i - 1]) / 2) / 2
        
    if np.ndim(x) == 1:
        x = x.reshape(-1, 1)
    if np.ndim(y) == 1:
        y = y.reshape(-1, 1)
    r, c = np.size(x), np.size(y)
    D0 = np.full((r + 1, c + 1),np.inf)
    D0[0,0]=0
    D1 = D0[1:, 1:]  # view
    C = D0.copy()
    #   求得D1为原始距离矩阵
    for i in range(r):
        for j in range(c):
            D1[i, j]=D0[i+1,j+1]=C[i+1,j+1]= np.abs(x[i]-y[j]) #曼哈顿城市距离
            C[i+1,j+1]+=min(C[i,j],C[i,j+1],C[i+1,j])   # 同时求得累计距离矩阵
    
    i, j = np.array(C.shape) - 2  #从倒数第二个数开始算路径
    path = [[i+1,j+1]]
    while (i > 0) or (j > 0):
        tb = np.argmin((C[i, j], C[i, j + 1], C[i + 1, j]))
        if tb == 0:
            path.append([i, j])
            i -= 1
            j -= 1
        elif tb == 1:
            path.append([i, j + 1])
            i -= 1
        else:  # (tb == 2):
            path.append([i + 1, j])
            j -= 1
    path.reverse()
    return C[-1,-1],C,path

# # basic-DTW----------------------------------------------------
# def Mydtw(x,y): #x做列，y做行
#     if np.ndim(x) == 1:
#         x = x.reshape(-1, 1)
#     if np.ndim(y) == 1:
#         y = y.reshape(-1, 1)
#     r, c = np.size(x), np.size(y)
#     D0 = np.full((r + 1, c + 1),np.inf)
#     D0[0,0]=0
#     D1 = D0[1:, 1:]  # view
#     C = D0.copy()
#     #   求得D1为原始距离矩阵
#     for i in range(r):
#         for j in range(c):
#             D1[i, j]=D0[i+1,j+1]=C[i+1,j+1]= np.abs(x[i]-y[j]) #曼哈顿城市距离
#             C[i+1,j+1]+=min(C[i,j],C[i,j+1],C[i+1,j])   # 同时求得累计距离矩阵
#     i, j = np.array(C.shape) - 2  #从倒数第二个数开始算路径
#     path = [[i+1,j+1]]
#     while (i > 0) or (j > 0):
#         tb = np.argmin((C[i, j], C[i, j + 1], C[i + 1, j]))
#         if tb == 0:
#             path.append([i, j])
#             i -= 1
#             j -= 1
#         elif tb == 1:
#             path.append([i, j + 1])
#             i -= 1
#         else:  # (tb == 2):
#             path.append([i + 1, j])
#             j -= 1
#     path.reverse()
#     return C[-1,-1],C,path
  
def Mydtw(x,y,window=1): #x做列，y做行
    if np.ndim(x) == 1:
        x = x.reshape(-1, 1)
    if np.ndim(y) == 1:
        y = y.reshape(-1, 1)
    r, c = np.size(x), np.size(y)
    D0 = np.zeros((r + 1, c + 1))
#     D0[0,0]=0
    D0[0,1:]=np.inf
    D0[1:,0]=np.inf
    D1 = D0[1:, 1:]  # view
    C = D0.copy()
    #   求得D1为原始距离矩阵
    for i in range(r):
        for j in range(int(max(i-np.ceil(window*len),0)),int(min(len,i+np.ceil(window*len)))):
            D1[i, j]=D0[i+1,j+1]=C[i+1,j+1]= np.abs(x[i]-y[j]) #曼哈顿城市距离
            C[i+1,j+1]+=min(C[i,j],C[i,j+1],C[i+1,j])   # 同时求得累计距离矩离
        print(D1[i,j])          
    i, j = np.array(C.shape) - 2  #从倒数第二个数开始算路径
    path = [[i+1,j+1]]
    while (i > 0) or (j > 0):
        tb = np.argmin((C[i, j], C[i, j + 1], C[i + 1, j]))
        if tb == 0:
            path.append([i, j])
            i -= 1
            j -= 1
        elif tb == 1:
            path.append([i, j + 1])
            i -= 1
        else:  # (tb == 2):
            path.append([i + 1, j])
            j -= 1
    path.reverse()
    return C[-1,-1],C,path
  
def reduce(x, n):

    x_shrinked = [(x[i] + x[1 + i])/n for i in range(0, np.size(x)-np.size(x) % n - 1, n)]
    x_shrinked = np.array(x_shrinked)
    return x_shrinked


def MyFastDtw_v3(x, y, n=1, window=1):
    len_x = np.size(x)
    if np.ndim(x) == 1:
        x = x.reshape(-1, 1)
    if np.ndim(y) == 1:
        y = y.reshape(-1, 1)
    # 二次采样
    x_half = reduce(x, n)
    y_half = reduce(y, n)
    r, c = np.size(x_half), np.size(y_half)
    D0 = np.full((r + 1, c + 1), np.inf)
    # D0 = np.zeros((r + 1, c + 1))
    D0[0, 0] = 0
    D1 = D0[1:, 1:]
    C = D0.copy()
    for i in range(r):
        for j in range(int(max(i - np.ceil(window * r), 0)), int(min(r, i + np.ceil(window * r)))):
            D1[i, j] = D0[i + 1, j + 1] = C[i + 1, j + 1] = np.abs(x[i] - y[j])  # 曼哈顿城市距离
            C[i + 1, j + 1] += min(C[i, j], C[i, j + 1], C[i + 1, j])  # 同时求得累计距离矩阵
    #         print(max(i-np.ceil(window*len)),min(len,i+np.ceil(window*len)))

    i, j = np.array(C.shape) - 2  # 从倒数第二个数开始算路径
    path = [[i + 1, j + 1]]
    while (i > 0) or (j > 0):
        tb = np.argmin((C[i, j], C[i, j + 1], C[i + 1, j]))
        if tb == 0:
            path.append([i, j])
            i -= 1
            j -= 1
        elif tb == 1:
            path.append([i, j + 1])
            i -= 1
        else:  # (tb == 2):
            path.append([i + 1, j])
            j -= 1
    path.reverse()
    return C[-1, -1], C, path


def LCSS(A,B):
    if np.ndim(A) == 1:
        A = A.reshape(-1, 1)
    if np.ndim(B) == 1:
        B = B.reshape(-1, 1)
    c,r = np.size(A),np.size(B)
    L = np.zeros((r+1,c+1))
    for i in range(1,r+1):
        for j in range(1,c+1):
            if abs(A[j-1] - B[i-1]) < 1e-3:
                L[i,j]=L[i-1,j-1]+1
            else:
                L[i,j]=max(L[i-1,j],L[i,j-1])
                # print(L[i, j])
    # print(L)
    dist_LCSS=1-L[-1,-1]/((c+r)/2)
    return dist_LCSS,L


# ---------------------定义的函数部分-----------------------------------------------------------------------------------------------

start=datetime.datetime.now()

# 测试用例
# dist,C,path=Mydtw(x,y)
# dist,C,path=MyFastDtw_v3(x,y,n=1,window=0.05)   #w=1,t=0.070s;w=0.4,t=0.044s;w=0.3,t=0.036,w=0.2,t=0.026
# dist,C,path=DDTW(x,y)
# dist,C,path = MyFastDtw_v3(x,y,n=5)
L_dist,L=LCSS(x,y)

# dist,C,path=DDTW(x,y)
# print(dist,'\n',C,'\n',path)
print(L_dist,L)
# plt.imshow(C[1:,1:],interpolation='nearest',cmap='jet',origin='lower')
# #根据像素绘制图片 origin表示渐变程度
# plt.colorbar()
# #显示像素与数据对比
# plt.xticks(())
# plt.yticks(())
# #不显示坐标轴刻度
# plt.show()
# #显示图片
end=datetime.datetime.now()
print('runing time=',end-start)

0.9583333333333334 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 6. 6. 6.]
 [0. 0. 0. ... 6. 6. 6.]
 [0. 0. 0. ... 6. 6. 6.]]
runing time= 0:00:00.060304
